In [32]:
import os
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from scipy import stats
from tqdm import tqdm_notebook as tqdm

np.random.seed(42)

import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

path = "./data/stability_data"
output_path = os.path.join(path, "stability_with_unirep_fusion.hdf")

ids = pd.read_hdf(output_path, key="ids")
reps = pd.read_hdf(output_path, key="reps")

print("X: {}".format(reps.shape))
print("Y: {}".format(ids["stability"].shape))

X: (11380, 5700)
Y: (11380,)


In [36]:
test_set_protein_names = ["hYAP65", "EHEE_rd2_0005", "HHH_rd3_0138", "HHH_rd2_0134", "villin", "EEHEE_rd3_1498", "EEHEE_rd3_0037", "HEEH_rd3_0872"]

test_proteins = pd.DataFrame(columns=["sequence", "stability", "name"])
test_reps = pd.DataFrame(columns=list(range(0, 5700)))
train_proteins = pd.DataFrame(columns=["sequence", "stability", "name"])
train_reps = pd.DataFrame(columns=list(range(0, 5700)))
# Iterate the ids and put them into train or test based on matching the 
for index, row in tqdm(ids.iterrows(), total=ids.shape[0]):
    if any(protein in row["name"] for protein in test_set_protein_names):
        test_proteins.loc[len(test_proteins)]=[row["sequence"], row["stability"], row["name"]]
        test_reps.loc[len(test_reps)]=reps.iloc[index]
    else:
        train_proteins.loc[len(train_proteins)]=[row["sequence"], row["stability"], row["name"]]
        train_reps.loc[len(train_reps)]=reps.iloc[index]

In [31]:
print(train_proteins.shape)
print(train_reps.shape)
print(test_proteins.shape)
print(test_reps.shape)
print(test_proteins.iloc[0])
print(test_reps.iloc[0])
print(ids.iloc[0])
print(reps.iloc[0])

(3614, 3)
(2271, 3)
sequence     TTIKVNGQEYTVPLSPEQAAKAAKKRWPDYEVQIHGNTVKVTR
stability                                           0.85
name                                  EEHEE_rd3_0037.pdb
Name: 0, dtype: object
0       0.013220
1      -0.023352
2       0.017943
3       0.001995
4      -0.179529
5       0.010197
6      -0.149218
7       0.002266
8      -0.018254
9       0.072948
10      0.207161
11      0.010123
12      0.064938
13      0.053157
14      0.016196
15      0.009681
16      0.045279
17     -0.035806
18      0.137569
19     -0.026070
20     -0.000789
21      0.050852
22      0.058471
23     -0.086977
24      0.056679
25     -0.037811
26     -0.009353
27      0.025263
28      0.010998
29     -0.052333
          ...   
5670    0.624806
5671   -0.312815
5672    1.221013
5673   -0.288682
5674   -0.381718
5675   -2.431110
5676   -1.886887
5677    2.433378
5678   -0.459735
5679   -1.152914
5680   -0.564261
5681    0.883261
5682    1.340542
5683   -0.711250
5684   -0.617897
5685

In [33]:
from data_utils import aa_seq_to_int

seq_ints = []

for seq in ids["sequence"]:
    seq_int = aa_seq_to_int(seq)
    seq_ints += [seq_int]
    
X_train, X_test, y_train, y_test = train_test_split(seq_ints, ids["stability"], test_size=0.15)

cv = 3
# LassoLars usage: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLars.html#sklearn.linear_model.LassoLars
reg_sequences = linear_model.LassoLarsCV(cv=cv)
print("Training...")
reg_sequences.fit(X_train, y_train)

score_train = reg_sequences.score(X_train, y_train)
score_test = reg_sequences.score(X_test, y_test)
print("Train score: {}".format(score_train))
print("Test score: {}".format(score_test))

Training...
Train score: 0.648888144394212
Test score: 0.6165686119198099


In [39]:
from data_utils import aa_seq_to_int

train_seq_ints = []
test_seq_ints = []

for seq in train_proteins["sequence"]:
    seq_int = aa_seq_to_int(seq)
    train_seq_ints += [seq_int]
    
for seq in test_proteins["sequence"]:
    seq_int = aa_seq_to_int(seq)
    test_seq_ints += [seq_int]
    
print(len(train_seq_ints))
print(len(test_seq_ints))

cv = 3
# LassoLars usage: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoLars.html#sklearn.linear_model.LassoLars
reg_sequences = linear_model.LassoLarsCV(cv=cv)
print("Training...")
reg_sequences.fit(train_seq_ints, train_proteins["stability"])

score_train = reg_sequences.score(train_seq_ints, train_proteins["stability"])
score_test = reg_sequences.score(test_seq_ints, test_proteins["stability"])
print("Train score: {}".format(score_train))
print("Test score: {}".format(score_test))

6784
4596
Training...
Train score: 0.0
Test score: -0.07532190932976013


In [40]:
X_train, X_test, y_train, y_test = train_test_split(reps, ids["stability"], test_size=0.15)
print("{} points in test set".format(X_train.shape[0]))

cv = 3
reg_uni = linear_model.LassoLarsCV(cv=cv)
print("Training {}-fold cross validated LassoLars with EclRep Fusion representations as input...".format(cv))
reg_uni.fit(X_train, y_train)

9673 points in test set
Training 3-fold cross validated LassoLars with EclRep Fusion representations as input...


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/least_angle.py:339: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 246 iterations, alpha=4.474e-05, previous alpha=4.474e-05, with an active set of 165 regressors.
  ConvergenceWarning)


LassoLarsCV(copy_X=True, cv=3, eps=2.220446049250313e-16, fit_intercept=True,
      max_iter=500, max_n_alphas=1000, n_jobs=1, normalize=True,
      positive=False, precompute='auto', verbose=False)

In [1]:
score_train = reg_uni.score(X_train, y_train)
score_test = reg_uni.score(X_test, y_test)
print("Train score: {}".format(score_train))
print("Test score: {}".format(score_test))

# Get Spearman's p correlation on test set predictions
test_predictions = reg_uni.predict(X_test)
spearman_test = stats.spearmanr(test_predictions, y_test)
print("Spearman's p on test set: {}".format(spearman_test))

# Plot the predictions vs. measured values
import plotly.plotly as py
import plotly
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username='xanderdunn', api_key='GtTpDQavToMaADqeMMu4')

trace = go.Scatter(
    x = test_predictions,
    y = y_test,
    mode = 'markers'
)

py.iplot([trace], filename="Peptide Stability Prediction vs. Measured Stability")

NameError: name 'reg_uni' is not defined

![Capture.PNG](/notebooks/Capture.PNG)

In [28]:
cv = 3
reg_uni = linear_model.LassoLarsCV(cv=cv)
print("Training {}-fold cross validated LassoLars with EclRep Fusion representations as input...".format(cv))
print(train_reps.shape)
reg_uni.fit(train_reps, train_proteins["stability"])

Training 3-fold cross validated LassoLars with EclRep Fusion representations as input...
(6784, 5700)


LassoLarsCV(copy_X=True, cv=3, eps=2.220446049250313e-16, fit_intercept=True,
      max_iter=500, max_n_alphas=1000, n_jobs=1, normalize=True,
      positive=False, precompute='auto', verbose=False)

In [29]:
score_train = reg_uni.score(train_reps, train_proteins["stability"])
score_test = reg_uni.score(test_reps, test_proteins["stability"])
print("Train score: {}".format(score_train))
print("Test score: {}".format(score_test))

Train score: -2.2648549702353197e-14
Test score: -0.07532198021384673
